In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
# print(os.getcwd())
# sys.path.append(os.getcwd())

import logging
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torchvision
import torch.nn as nn
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())

PyTorch version: 2.0.0+cu118
Torchvision version: 0.15.1+cu118
CUDA is available: True


In [2]:
datadir= Path('/dlab/ldrive/CBT/USLJ-DSDE_DATA-I10008/BenchmarkDatasets/hpa-single-cell-image-classification')
train_dataset_dir = datadir.joinpath('train')
train_csv = datadir.joinpath('train_select_1K.csv')

In [3]:
from func.HPASCDataset import HPASCDataset, TrDataset
from func.trainer import load_train_objs
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, ToTensor, Resize, v2
input_ch_ct = 4
debug_size = 100
n_class = 19

tf_dataset = Compose(
    [
        ToTensor(), 
        Resize(size = (1024, 1024), antialias = True),
    ]

)
HPA_dataset = HPASCDataset(
                        input_csv = train_csv, 
                        root = train_dataset_dir, 
                        input_ch_ct = input_ch_ct,
                        n_class = n_class, 
                        debug_size = debug_size,
                        transform = tf_dataset
                        )

dataloader = DataLoader(
                    HPA_dataset,
                    batch_size=64,
                    pin_memory=True,
                    num_workers=32,
                    shuffle=False)

/dlab/ldrive/CBT/USLJ-DSDE-I10007/DSDE/runtime_env/miniconda3/envs/mspytorch/lib/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/dlab/ldrive/CBT/USLJ-DSDE-I10007/DSDE/runtime_env/miniconda3/envs/mspytorch/lib/python3.9/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major break

In [4]:
def get_mean_and_std(loader):
    mean = 0.
    std = 0.
    total_images_count = 0
    for data in tqdm(loader):
        images = data['image']
        image_count_in_a_batch = images.size(0)
        # print(images.shape)
        images = images.view(image_count_in_a_batch, images.size(1), -1)
        # print(images.shape)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += image_count_in_a_batch
    
    mean /= total_images_count
    std /= total_images_count
    return mean, std

In [5]:
mean_100, std_100 = get_mean_and_std(dataloader)
print(mean_100)
print(std_100)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:24<00:00, 12.41s/it]

tensor([0.0516, 0.0472, 0.0755, 0.0821])
tensor([0.1351, 0.0812, 0.1244, 0.1252])


In [6]:
HPA_dataset = HPASCDataset(
                        input_csv = train_csv, 
                        root = train_dataset_dir, 
                        input_ch_ct = input_ch_ct,
                        n_class = n_class, 
                        transform = tf_dataset
                        )

dataloader = DataLoader(
                    HPA_dataset,
                    batch_size=64,
                    pin_memory=True,
                    num_workers=32,
                    shuffle=False)

mean_1k, std_1k = get_mean_and_std(dataloader)
print(mean_1k)
print(std_1k)

NameError: name 'transform_dataset' is not defined

In [ ]:
train_csv = datadir.joinpath('train.csv')

HPA_dataset = HPASCDataset(
                        input_csv = train_csv, 
                        root = train_dataset_dir, 
                        input_ch_ct = input_ch_ct,
                        n_class = n_class,
                        debug_size = 3000, 
                        transform = tf_dataset
                        )

dataloader = DataLoader(
                    HPA_dataset,
                    batch_size=128,
                    pin_memory=True,
                    num_workers=32,
                    shuffle=False)

mean_full, std_full = get_mean_and_std(dataloader)

100%|██████████| 24/24 [04:01<00:00, 10.08s/it]


In [ ]:
print(mean_100)
print(std_100)
print(mean_1k)
print(std_1k)
print(mean_full)
print(std_full)

tensor([0.0496, 0.0483, 0.0711, 0.0789])
tensor([0.1344, 0.0795, 0.1200, 0.1196])
tensor([0.0480, 0.0442, 0.0724, 0.0723])
tensor([0.1302, 0.0767, 0.1202, 0.1153])
tensor([0.0540, 0.0530, 0.0804, 0.0806])
tensor([0.1420, 0.0831, 0.1272, 0.1229])
